In [3]:
import pickle, os
import cv2
import numpy as np
with open("../data/AID_split.pkl", 'rb') as f:
    split = pickle.load(f)

data_path = "/root/autodl-tmp/SRDataset/AID"
rslt_path = "/root/autodl-tmp/SRDataset/AID_split"
import tqdm

In [4]:
for split_name, split_list in split.items():
    hr_folder = os.path.join(rslt_path,split_name, "HR")

    if not os.path.exists(hr_folder):
        os.makedirs(hr_folder)

    for filename in tqdm.tqdm(split_list, desc=f"preparing {split_name}"):
        file_path = os.path.join(data_path, filename)
        hr = cv2.imread(file_path)
        hr = cv2.cvtColor(hr, cv2.COLOR_BGR2RGB)
        _filename = os.path.basename(filename).replace(".jpg", ".png")
        _hr_path = os.path.join(hr_folder, _filename)
        cv2.imwrite(_hr_path, hr)

preparing train:   0%|          | 0/8000 [00:00<?, ?it/s]

preparing test: 100%|██████████| 1000/1000 [00:23<00:00, 42.72it/s]


In [3]:
from PIL import Image
def resize_pillow(img, scale=None, size=None, mode="bicubic"):
    assert scale or size
    if type(img) == np.ndarray:
        h, w = img.shape[0:2]
    else:
        h, w = img.size[0:2]
    if not size:
        size = (int(h * scale), int(w * scale))

    size = (size[1], size[0])
    to_numpy_flag = 0
    if type(img) == np.ndarray:
        to_numpy_flag = 1
        img = Image.fromarray(img)

    interpolation = {
        "bicubic": Image.BICUBIC,
        "bilinear": Image.BILINEAR,
        "nearest": Image.NEAREST,
    }[mode]

    img = img.resize(size, resample=interpolation)
    if to_numpy_flag:
        img = np.array(img)

    return img

In [8]:
scale = 4
for split_name, split_list in split.items():
    split_path = os.path.join(rslt_path, split_name)
    lr_folder = os.path.join(split_path, "LR")
    hr_folder = os.path.join(split_path, "HR")

    if not os.path.exists(lr_folder):
        os.makedirs(lr_folder)
    if not os.path.exists(hr_folder):
        os.makedirs(hr_folder)

    for filename in tqdm.tqdm(split_list, desc=f"preparing {split_name}"):
        file_path = os.path.join(data_path, filename)
        hr = cv2.imread(file_path)
        hr = cv2.cvtColor(hr, cv2.COLOR_BGR2RGB)

        # resize
        lr = resize_pillow(hr, 1/4)

        _filename = os.path.basename(filename).replace(".jpg", ".png")
    
        _lr_path = os.path.join(lr_folder, _filename)
        _hr_path = os.path.join(hr_folder, _filename)

        cv2.imwrite(_lr_path, lr)
        cv2.imwrite(_hr_path, hr)

preparing test: 100%|██████████| 1000/1000 [00:42<00:00, 23.52it/s]
